## Imports

In [2]:
from scipy import signal
from scipy.stats import pearsonr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
from sklearn.decomposition import PCA
import seaborn as sb
from scipy.sparse.linalg import eigsh
from sklearn.decomposition import PCA
from scipy import io
from scipy.sparse.linalg import eigsh

## Load and prepare Data

In [ ]:
# from https://figshare.com/articles/dataset/Recordings_of_ten_thousand_neurons_in_visual_cortex_in_response_to_2_800_natural_images/6845348
mat_file_path = 'calcium_data/V1/natimg2800_M160825_MP027_2016-12-14.mat'

mat_data = scipy.io.loadmat(mat_file_path)

### stimulus responses
resp = mat_data[‘stim’][0]['resp'][0]    # stimuli by neurons
istim = mat_data[‘stim’][0]['istim'][0]   # identities of stimuli in resp
spont = at_datat[‘stim’][0]['spont'][0]  # timepoints by neurons

### cell information
med = mat_data[‘med’]                 # cell centers (X Y Z)
mat_data[‘stat’][0]     # first cell’s stats
mat_data[‘stat’][0][‘npix’]       # one example field, tells you how pixels make up the cell

### loading images
mt = sio.loadmat('images_natimg2800_all.mat')
imgs = mt['imgs']  # 68 by 270 by number of images
# check out first image using matplotlib.pyplot
plt.imshow(imgs[:,:,0])

## Utils

In [3]:
def get_powerlaw(ss, trange):
    ''' fit exponent to variance curve'''
    logss = np.log(np.abs(ss))
    y = logss[trange][:,np.newaxis]
    trange += 1
    nt = trange.size
    x = np.concatenate((-np.log(trange)[:,np.newaxis], np.ones((nt,1))), axis=1)
    w = 1.0 / trange.astype(np.float32)[:,np.newaxis]
    b = np.linalg.solve(x.T @ (x * w), (w * x).T @ y).flatten()
    
    allrange = np.arange(0, ss.size).astype(int) + 1
    x = np.concatenate((-np.log(allrange)[:,np.newaxis], np.ones((ss.size,1))), axis=1)
    ypred = np.exp((x * b).sum(axis=1))
    alpha = b[0]
    return alpha,ypred

def shuff_cvPCA(X, nshuff=5):
    ''' X is 2 x stimuli x neurons '''
    nc = min(1024, X.shape[1])
    ss=np.zeros((nshuff,nc))
    for k in range(nshuff):
        iflip = np.random.rand(X.shape[1]) > 0.5
        X0 = X.copy()
        X0[0,iflip] = X[1,iflip]
        X0[1,iflip] = X[0,iflip]
        ss[k]=cvPCA(X0)
    return ss

def cvPCA(X):
    ''' X is 2 x stimuli x neurons '''
    pca = PCA(n_components=min(1024, X.shape[1])).fit(X[0].T)
    u = pca.components_.T
    sv = pca.singular_values_
    
    xproj = X[0].T @ (u / sv)
    cproj0 = X[0] @ xproj
    cproj1 = X[1] @ xproj
    ss = (cproj0 * cproj1).sum(axis=0)
    return ss

## Powerlaw analysis

In [4]:
dat = io.loadmat('calcium_data/V1/natimg2800_M170714_MP032_2017-09-14.mat')

resp = dat['stim'][0]['resp'][0] # stim x neurons
spont = dat['stim'][0]['spont'][0] # timepts x neurons
istim = (dat['stim'][0]['istim'][0]).astype(np.int32) # stim ids 
istim -= 1 # get out of MATLAB convention
istim = istim[:,0]
nimg = istim.max() # these are blank stims (exclude them)
resp = resp[istim<nimg, :]
istim = istim[istim<nimg]

# subtract spont (32D)
mu = spont.mean(axis=0)
sd = spont.std(axis=0) + 1e-6
resp = (resp - mu) / sd
spont = (spont - mu) / sd
sv,u = eigsh(spont.T @ spont, k=32)
resp = resp - (resp @ u) @ u.T

# mean center each neuron
resp -= resp.mean(axis=0)

In [5]:
### sanity check - decent signal variance ?
# split stimuli into two repeats
NN = resp.shape[1]
sresp = np.zeros((2, nimg, NN), np.float64)
inan = np.zeros((nimg,), np.bool)
for n in range(nimg):
    ist = (istim==n).nonzero()[0]
    i1 = ist[:int(ist.size/2)]
    i2 = ist[int(ist.size/2):]
    # check if two repeats of stim
    if np.logical_or(i2.size < 1, i1.size < 1):
        inan[n] = 1
    else:
        sresp[0, n, :] = resp[i1, :].mean(axis=0)
        sresp[1, n, :] = resp[i2, :].mean(axis=0)
        
# remove image responses without two repeats
sresp = sresp[:,~inan,:]
    
snorm = sresp - sresp.mean(axis=1)[:,np.newaxis,:]
snorm = snorm / (snorm.std(axis=1)[:,np.newaxis,:] + 1e-6)

cc = (snorm[0].T @ snorm[1]) / sresp.shape[1]

print('fraction of signal variance: %2.3f'%np.diag(cc).mean())

C:\Users\Afernandez\AppData\Local\Temp\ipykernel_30000\1566601546.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  inan = np.zeros((nimg,), np.bool)


fraction of signal variance: 0.165


In [6]:
### sanity check - decent decoding ?
# 1 nearest neighbor decoder    
# (mean already subtracted)
cc = sresp[0] @ sresp[1].T
cc /= (sresp[0]**2).sum()
cc /= (sresp[1]**2).sum()
nstims = sresp.shape[1]
print('decoding accuracy: %2.3f'%(cc.argmax(axis=1)==np.arange(0,nstims,1,int)).mean())

decoding accuracy: 0.412


In [ ]:
### sanity check - is the powerlaw close to 1 ?
# powerlaw

# compute cvPCA
ss = shuff_cvPCA(sresp)

# compute powerlaw of averaged shuffles
ss = ss.mean(axis=0)
alpha,ypred = get_powerlaw(ss/ss.sum(), np.arange(11,5e2).astype(int))  
print('powerlaw, alpha=%2.3f'%alpha)

plt.loglog(np.arange(0,ss.size)+1, ss/ss.sum())
plt.loglog(np.arange(0,ss.size)+1, ypred, c='k')
plt.show()

In [7]:
sresp.shape

(2, 2800, 9476)

In [8]:
data = sresp[0]

In [9]:
data.shape

(2800, 9476)

In [13]:
data[4,0]

0.6080321073532104

## Pairwise correlations

In [16]:
def compute_pairwise_correlations(data):
    """
    Compute the overall pairwise correlation level for each session of each animal.

    Parameters:
    data (dict): A nested dictionary where the first key is the animal ID and the second key is the session ID, 
                 mapping to a data matrix for that session.
    p_value_threshold (float): The threshold for considering correlations as significant.

    Returns:
    correlations (dict): A nested dictionary with the mean correlation for each session of each animal.
    """
    n_neurons = data.shape[1]
    correlation = []
    for i in range(n_neurons):
        print(str(i)+'/'+str(n_neurons))
        for j in range(i+1, n_neurons):
            corr, p_value = pearsonr(data[:, i], data[:, j])
            correlation.append(corr)

    return correlations

In [17]:
correlations = compute_pairwise_correlations(data)

0/9476
1/9476
2/9476
3/9476
4/9476
5/9476
6/9476
7/9476
8/9476
9/9476
10/9476
11/9476
12/9476
13/9476
14/9476
15/9476
16/9476
17/9476
18/9476
19/9476
20/9476
21/9476
22/9476
23/9476
24/9476
25/9476
26/9476
27/9476
28/9476
29/9476
30/9476
31/9476
32/9476
33/9476
34/9476
35/9476
36/9476
37/9476
38/9476
39/9476
40/9476
41/9476
42/9476
43/9476
44/9476
45/9476
46/9476
47/9476
48/9476
49/9476
50/9476
51/9476
52/9476
53/9476
54/9476
55/9476
56/9476
57/9476
58/9476
59/9476
60/9476
61/9476
62/9476
63/9476
64/9476
65/9476
66/9476
67/9476
68/9476
69/9476
70/9476
71/9476
72/9476
73/9476
74/9476
75/9476
76/9476
77/9476
78/9476
79/9476
80/9476
81/9476
82/9476
83/9476
84/9476
85/9476
86/9476
87/9476
88/9476
89/9476
90/9476
91/9476
92/9476
93/9476
94/9476
95/9476
96/9476
97/9476
98/9476
99/9476
100/9476
101/9476
102/9476
103/9476
104/9476
105/9476
106/9476
107/9476
108/9476
109/9476
110/9476
111/9476
112/9476
113/9476
114/9476
115/9476
116/9476
117/9476
118/9476
119/9476
120/9476
121/9476
122/9476
123


KeyboardInterrupt



In [ ]:
plt.figure(figsize=(10,4))

plt.hist(correlations, bins=200, color='k', alpha=1, edgecolor='white', linewidth=.5)

plt.axvline(x=np.mean(correlations), color='white', linestyle='--')

plt.xlabel('Corr. coef. (ρ)', fontsize=18)
plt.ylabel('No. of neuron pairs', fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.xlim(-0.15, 0.5)

sb.despine()
#plt.savefig('plots/pairwiseCorrDistr.png', dpi=600)
plt.show()